The goal of this assignment is to develop a Feedforward neural network for topic classification. 



For that purpose, you will implement:

- Text processing methods for transforming raw text data into input vectors for your network  (**1 mark**)


- A Feedforward network consisting of:
    - **One-hot** input layer mapping words into an **Embedding weight matrix** (**1 mark**)
    - **One hidden layer** computing the mean embedding vector of all words in input followed by a **ReLU activation function** (**1 mark**)
    - **Output layer** with a **softmax** activation. (**1 mark**)


- The Stochastic Gradient Descent (SGD) algorithm with **back-propagation** to learn the weights of your Neural network. Your algorithm should:
    - Use (and minimise) the **Categorical Cross-entropy loss** function (**1 mark**)
    - Perform a **Forward pass** to compute intermediate outputs (**2 marks**)
    - Perform a **Backward pass** to compute gradients and update all sets of weights (**3 marks**)
    - Implement and use **Dropout** after each hidden layer for regularisation (**1 marks**)



- Discuss how did you choose hyperparameters? You can tune the learning rate (hint: choose small values), embedding size {e.g. 50, 300, 500} and the dropout rate {e.g. 0.2, 0.5}. Please use tables or graphs to show training and validation performance for each hyperparameter combination  (**2 marks**). 



- After training a model, plot the learning process (i.e. training and validation loss in each epoch) using a line plot and report accuracy. Does your model overfit, underfit or is about right? (**1 mark**).



- Re-train your network by using pre-trained embeddings ([GloVe](https://nlp.stanford.edu/projects/glove/)) trained on large corpora. Instead of randomly initialising the embedding weights matrix, you should initialise it with the pre-trained weights. During training, you should not update them (i.e. weight freezing) and backprop should stop before computing gradients for updating embedding weights. Report results by performing hyperparameter tuning and plotting the learning process. Do you get better performance? (**1 marks**).



- Extend you Feedforward network by adding more hidden layers (e.g. one more or two). How does it affect the performance? Note: You need to repeat hyperparameter tuning, but the number of combinations grows exponentially. Therefore, you need to choose a subset of all possible combinations (**3 marks**)


- Provide well documented and commented code describing all of your choices. In general, you are free to make decisions about text processing (e.g. punctuation, numbers, vocabulary size) and hyperparameter values. We expect to see justifications and discussion for all of your choices. You must provide detailed explanations of your implementation, provide a detailed analysis of the results (e.g. why a model performs better than other models etc.) including error analyses (e.g. examples and discussion/analysis of missclasifications etc.)  (**10 marks**). 



- Provide efficient solutions by using Numpy arrays when possible. Executing the whole notebook with your code should not take more than 10 minutes on any standard computer (e.g. Intel Core i5 CPU, 8 or 16GB RAM) excluding hyperparameter tuning runs and loading the pretrained vectors. You can find tips in Lab 1 (**2 marks**). 



### Data 

The data you will use for the task is a subset of the [AG News Corpus](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) and you can find it in the `./data_topic` folder in CSV format:

- `data_topic/train.csv`: contains 2,400 news articles, 800 for each class to be used for training.
- `data_topic/dev.csv`: contains 150 news articles, 50 for each class to be used for hyperparameter selection and monitoring the training process.
- `data_topic/test.csv`: contains 900 news articles, 300 for each class to be used for testing.

Class 1: Politics, Class 2: Sports, Class 3: Economy

### Pre-trained Embeddings

You can download pre-trained GloVe embeddings trained on Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download) from [here](http://nlp.stanford.edu/data/glove.840B.300d.zip). No need to unzip, the file is large.

### Save Memory

To save RAM, when you finish each experiment you can delete the weights of your network using `del W` followed by Python's garbage collector `gc.collect()`




### Submission Instructions

You **must** submit a Jupyter Notebook file (assignment_yourusername.ipynb) and an exported PDF version (you can do it from Jupyter: `File->Download as->PDF via Latex`, you need to have a Latex distribution installed e.g. MikTex or MacTex and pandoc). If you are unable to export the pdf via Latex, you can print the notebook web page to a pdf file from your browser (e.g. on Firefox: File->Print->Save to PDF).


You are advised to follow the code structure given in this notebook by completing all given funtions. You can also write any auxilliary/helper functions (and arguments for the functions) that you might need but note that you can provide a full solution without any such functions. Similarly, you can just use only the packages imported below but you are free to use any functionality from the [Python Standard Library](https://docs.python.org/3/library/index.html), NumPy, SciPy (excluding built-in softmax funtcions) and Pandas. You are **not allowed to use any third-party library** such as Scikit-learn (apart from metric functions already provided), NLTK, Spacy, Keras, Pytorch etc.. You should mention if you've used Windows to write and test your code because we mostly use Unix based machines for marking (e.g. Ubuntu, MacOS). 

There is no single correct answer on what your accuracy should be, but correct implementations usually achieve F1-scores around 80\% or higher. The quality of the analysis of the results and discussion is as important as the implementation and accuracy of your models. Please be brief and consice in your discussion and analyses. 

This assignment will be marked out of 30. It is worth 30\% of your final grade in the module.

The deadline for this assignment is **23:59 on Mon, 26 Apr 2023** and it needs to be submitted via Blackboard. Standard departmental penalties for lateness will be applied. We use a range of strategies to **detect [unfair means](https://www.sheffield.ac.uk/ssid/unfair-means/index)**, including Turnitin which helps detect plagiarism. Use of unfair means would result in getting a failing grade.



In [92]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
from time import localtime, strftime
from scipy.stats import spearmanr,pearsonr
import zipfile
import gc

# fixing random seed for reproducibility
random.seed(123)
np.random.seed(123)


## Transform Raw texts into training and development data

First, you need to load the training, development and test sets from their corresponding CSV files (tip: you can use Pandas dataframes).

In [93]:
# 读取csv文件并将其存储为Pandas DataFrame
train_df = pd.read_csv('data_topic/train.csv',header=None)
test_df = pd.read_csv('data_topic/test.csv',header=None)
dev_df = pd.read_csv('data_topic/dev.csv',header=None)

train_data = np.array(train_df)
test_data = np.array(test_df)
dev_data = np.array(dev_df)

# Create input representations


To train your Feedforward network, you first need to obtain input representations given a vocabulary. One-hot encoding requires large memory capacity. Therefore, we will instead represent documents as lists of vocabulary indices (each word corresponds to a vocabulary index). 


## Text Pre-Processing Pipeline

To obtain a vocabulary of words. You should: 
- tokenise all texts into a list of unigrams (tip: you can re-use the functions from Assignment 1) 
- remove stop words (using the one provided or one of your preference) 
- remove unigrams appearing in less than K documents
- use the remaining to create a vocabulary of the top-N most frequent unigrams in the entire corpus.


In [94]:
stop_words = ['a','in','on','at','and','or', 
              'to', 'the', 'of', 'an', 'by', 
              'as', 'is', 'was', 'were', 'been', 'be', 
              'are','for', 'this', 'that', 'these', 'those', 'you', 'i', 'if',
             'it', 'he', 'she', 'we', 'they', 'will', 'have', 'has',
              'do', 'did', 'can', 'could', 'who', 'which', 'what',
              'but', 'not', 'there', 'no', 'does', 'not', 'so', 've', 'their',
             'his', 'her', 'they', 'them', 'from', 'with', 'its']


### Unigram extraction from a document

You first need to implement the `extract_ngrams` function. It takes as input:
- `x_raw`: a string corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams you want to extract, e.g. (1,2) denotes extracting unigrams and bigrams.
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that data is already tokenised so you could opt for a simple white space tokenisation.
- `stop_words`: a list of stop words
- `vocab`: a given vocabulary. It should be used to extract specific features.

and returns:

- a list of all extracted features.


In [95]:
def extract_ngrams(x_raw, ngram_range=(1,3), token_pattern=r'\b[A-Za-z][A-Za-z]+\b', 
                   stop_words=[], vocab=set()):
    # Tokenize the input text using the given regular expression
    tokens = re.findall(token_pattern, x_raw.lower())
    # Remove stop words
    if stop_words:
        tokens = [token for token in tokens if token not in stop_words]
    # Extract n-grams of the specified range
    ngrams = []
    for n in range(ngram_range[0], ngram_range[1]+1):
        for i in range(len(tokens)-n+1):
            ngrams.append(' '.join(tokens[i:i+n]))
    # Keep only n-grams present in the given vocabulary (if any)
    if vocab:
        ngrams = [ngram for ngram in ngrams if ngram in vocab]
        
    return ngrams

### Create a vocabulary of n-grams

Then the `get_vocab` function will be used to (1) create a vocabulary of ngrams; (2) count the document frequencies of ngrams; (3) their raw frequency. It takes as input:
- `X_raw`: a list of strings each corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams you want to extract, e.g. (1,2) denotes extracting unigrams and bigrams.
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that data is already tokenised so you could opt for a simple white space tokenisation.
- `stop_words`: a list of stop words
- `min_df`: keep ngrams with a minimum document frequency.
- `keep_topN`: keep top-N more frequent ngrams.

and returns:

- `vocab`: a set of the n-grams that will be used as features.
- `df`: a Counter (or dict) that contains ngrams as keys and their corresponding document frequency as values.
- `ngram_counts`: counts of each ngram in vocab


In [96]:
def get_vocab(X_raw, ngram_range=(1,3), token_pattern=r'\b[A-Za-z][A-Za-z]+\b', 
              min_df=0, keep_topN=0, stop_words=[]):
    # Extract ngrams from each document
    ngrams_list = [extract_ngrams(x_raw, ngram_range, token_pattern, stop_words) for x_raw in X_raw]

    # Flatten the list of ngrams
    ngrams_flat = [ngram for ngrams in ngrams_list for ngram in ngrams]

    # Compute document frequency of each ngram
    df = Counter(ngrams_flat)

    # Filter ngrams based on min_df and keep_topN
    if min_df > 0:
        df = {ngram: freq for ngram, freq in df.items() if freq >= min_df}
    if keep_topN > 0:
        df = dict(sorted(df.items(), key=lambda item: item[1], reverse=True)[:keep_topN])

    # Create the vocabulary
    vocab = set(df.keys())

    # Compute ngram counts for each document
    ngram_counts = []
    for ngrams in ngrams_list:
        counts = Counter(ngrams)
        ngram_counts.append({ngram: counts[ngram] for ngram in vocab})

    return vocab, df, ngram_counts

Now you should use `get_vocab` to create your vocabulary and get document and raw frequencies of unigrams:

In [97]:
X_raw = ['This is the first document.', 'This is the second document.', 'And this is the third one.', 'Is this the first document?']

# 调用 get_vocab 函数创建词汇表和获取单词的文档频率和原始频率
vocab, df, ngram_counts = get_vocab(X_raw, ngram_range=(1,2), min_df=1, keep_topN=0, stop_words=stop_words)

# 输出词汇表
print('Vocabulary:', vocab)

# 输出单词的文档频率
print('Document frequency:', df)

# 输出单词的原始频率
raw_freqs = Counter([ngram for ngram_list in ngram_counts for ngram in ngram_list])
print('Raw frequency:', raw_freqs)

Vocabulary: {'first document', 'one', 'third one', 'document', 'third', 'second', 'second document', 'first'}
Document frequency: {'first': 2, 'document': 3, 'first document': 2, 'second': 1, 'second document': 1, 'third': 1, 'one': 1, 'third one': 1}
Raw frequency: Counter({'first document': 4, 'one': 4, 'third one': 4, 'document': 4, 'third': 4, 'second': 4, 'second document': 4, 'first': 4})


Then, you need to create vocabulary id -> word and word -> vocabulary id dictionaries for reference:

In [98]:
# 创建 id -> word 和 word -> id 字典
id2word = dict(enumerate(vocab))
word2id = {v: k for k, v in id2word.items()}

# 创建数据框并输出
df_vocab = pd.DataFrame({'id': list(id2word.keys()), 'word': list(id2word.values())})
df_vocab.set_index('id', inplace=True)
print('Vocabulary:')
print(df_vocab)

df_df = pd.DataFrame({'document frequency': df})
df_df.index.name = 'word'
print('\nDocument frequency:')
print(df_df)

df_rf = pd.DataFrame({'raw frequency': raw_freqs}, index=['ngrams'])
print('\nRaw frequency:')
print(df_rf)


Vocabulary:
               word
id                 
0    first document
1               one
2         third one
3          document
4             third
5            second
6   second document
7             first

Document frequency:
                 document frequency
word                               
document                          3
first                             2
first document                    2
one                               1
second                            1
second document                   1
third                             1
third one                         1

Raw frequency:
        raw frequency
ngrams            NaN


### Convert the list of unigrams  into a list of vocabulary indices

Storing actual one-hot vectors into memory for all words in the entire data set is prohibitive. Instead, we will store word indices in the vocabulary and look-up the weight matrix. This is equivalent of doing a dot product between an one-hot vector and the weight matrix. 

First, represent documents in train, dev and test sets as lists of words in the vocabulary:

In [99]:
# Convert train, dev, and test sets to lists of strings
train_docs = [doc[1] for doc in train_data]
test_docs = [doc[1] for doc in test_data]
dev_docs = [doc[1] for doc in dev_data]
train_labels = [int(doc[0]) for doc in train_data]
test_labels = [int(doc[0]) for doc in test_data]
dev_labels = [int(doc[0]) for doc in dev_data]

# Combine train, dev, and test sets into a single list
all_docs = train_docs + test_docs + dev_docs

# Create the vocabulary
vocab,df,ngram_counts = get_vocab(all_docs, ngram_range=(1,1), min_df=1, keep_topN=10000)



Then convert them into lists of indices in the vocabulary:

In [100]:
def docs_to_indices(docs, vocab):
    # Create a dictionary that maps each vocabulary word to its index
    word_to_index = {word: i for i, word in enumerate(vocab)}
    # Convert each document to a list of indices
    indices_list = [[word_to_index[word] for word in doc.split() if word in vocab] for doc in docs]
    return indices_list

# Convert the train, dev, and test sets to index lists
train_indices = docs_to_indices(train_docs, vocab)
dev_indices = docs_to_indices(dev_docs, vocab)
test_indices = docs_to_indices(test_docs, vocab)


Put the labels `Y` for train, dev and test sets into arrays: 

In [101]:
train_labels = np.array(train_labels)
dev_labels = np.array(dev_labels)
test_labels = np.array(test_labels)

# Network Architecture

Your network should pass each word index into its corresponding embedding by looking-up on the embedding matrix and then compute the first hidden layer $\mathbf{h}_1$:

$$\mathbf{h}_1 = \frac{1}{|x|}\sum_i W^e_i, i \in x$$

where $|x|$ is the number of words in the document and $W^e$ is an embedding matrix $|V|\times d$, $|V|$ is the size of the vocabulary and $d$ the embedding size.

Then $\mathbf{h}_1$ should be passed through a ReLU activation function:

$$\mathbf{a}_1 = relu(\mathbf{h}_1)$$

Finally the hidden layer is passed to the output layer:


$$\mathbf{y} = \text{softmax}(\mathbf{a}_1W) $$ 
where $W$ is a matrix $d \times |{\cal Y}|$, $|{\cal Y}|$ is the number of classes.

During training, $\mathbf{a}_1$ should be multiplied with a dropout mask vector (elementwise) for regularisation before it is passed to the output layer.

You can extend to a deeper architecture by passing a hidden layer to another one:

$$\mathbf{h_i} = \mathbf{a}_{i-1}W_i $$

$$\mathbf{a_i} = relu(\mathbf{h_i}) $$



# Network Training

First we need to define the parameters of our network by initiliasing the weight matrices. For that purpose, you should implement the `network_weights` function that takes as input:

- `vocab_size`: the size of the vocabulary
- `embedding_dim`: the size of the word embeddings
- `hidden_dim`: a list of the sizes of any subsequent hidden layers. Empty if there are no hidden layers between the average embedding and the output layer 
- `num_classes`: the number of the classes for the output layer

and returns:

- `W`: a dictionary mapping from layer index (e.g. 0 for the embedding matrix) to the corresponding weight matrix initialised with small random numbers (hint: use numpy.random.uniform with from -0.1 to 0.1)

Make sure that the dimensionality of each weight matrix is compatible with the previous and next weight matrix, otherwise you won't be able to perform forward and backward passes. Consider also using np.float32 precision to save memory.

In [103]:
def network_weights(vocab_size=1000, embedding_dim=300, 
                    hidden_dim=[], num_classes=3, init_val = 0.5):
    
    # Initialize weights for the embedding layer
    W = {0: np.random.uniform(-init_val, init_val, size=(vocab_size, embedding_dim)).astype(np.float32)}

    # Initialize weights for any subsequent hidden layers
    for i, dim in enumerate(hidden_dim):
        prev_dim = hidden_dim[i-1] if i > 0 else embedding_dim
        W[i+1] = np.random.uniform(-init_val, init_val, size=(prev_dim, dim)).astype(np.float32)

    # Initialize weights for the output layer
    prev_dim = hidden_dim[-1] if len(hidden_dim) > 0 else embedding_dim
    W[len(hidden_dim)+1] = np.random.uniform(-init_val, init_val, size=(prev_dim, num_classes)).astype(np.float32)
    return W
    

In [104]:
W = network_weights(vocab_size=3,embedding_dim=4,hidden_dim=[2], num_classes=2)
print(W)

{0: array([[ 0.19646919, -0.21386066, -0.27314854,  0.05131477],
       [ 0.21946897, -0.07689354,  0.4807642 ,  0.18482974],
       [-0.0190681 , -0.10788248, -0.15682198,  0.22904971]],
      dtype=float32), 1: array([[-0.06142776, -0.4403221 ],
       [-0.10195574,  0.2379954 ],
       [-0.31750828, -0.32454824],
       [ 0.03155137,  0.03182759]], dtype=float32), 2: array([[0.13440096, 0.34943178],
       [0.22445533, 0.11102351]], dtype=float32)}


Then you need to develop a `softmax` function (same as in Assignment 1) to be used in the output layer. 

It takes as input `z` (array of real numbers) and returns `sig` (the softmax of `z`)



In [105]:
def softmax(z):
    exp_z = np.exp(z)
    sig = exp_z / np.sum(exp_z)
    return sig


Now you need to implement the categorical cross entropy loss by slightly modifying the function from Assignment 1 to depend only on the true label `y` and the class probabilities vector `y_preds`:


In [106]:
def categorical_loss(y, y_preds):
    m = y.shape[0]
    l = -(1/m) * np.sum(y * np.log(y_preds + 1e-8)) 
    return l

Then, implement the `relu` function to introduce non-linearity after each hidden layer of your network 
(during the forward pass): 

$$relu(z_i)= max(z_i,0)$$

and the `relu_derivative` function to compute its derivative (used in the backward pass):

  
  relu_derivative($z_i$)=0, if $z_i$<=0, 1 otherwise.
  


Note that both functions take as input a vector $z$ 

Hint use .copy() to avoid in place changes in array z

In [107]:
def relu(z):
    a = np.maximum(z, 0)
    return a

def relu_derivative(z):
    dz = np.array(z, copy=True)
    dz[dz <= 0] = 0
    dz[dz > 0] = 1
    return dz

During training you should also apply a dropout mask element-wise after the activation function (i.e. vector of ones with a random percentage set to zero). The `dropout_mask` function takes as input:

- `size`: the size of the vector that we want to apply dropout
- `dropout_rate`: the percentage of elements that will be randomly set to zeros

and returns:

- `dropout_vec`: a vector with binary values (0 or 1)

In [108]:
def dropout_mask(size, dropout_rate):
    dropout_vec = np.random.binomial(1, 1-dropout_rate, size=size)
    return dropout_vec
    

In [109]:
print(dropout_mask(10, 0.2))
print(dropout_mask(10, 0.2))

[1 1 1 1 1 1 1 1 1 1]
[1 1 1 0 0 1 1 1 1 1]


Now you need to implement the `forward_pass` function that passes the input x through the network up to the output layer for computing the probability for each class using the weight matrices in `W`. The ReLU activation function should be applied on each hidden layer. 

- `x`: a list of vocabulary indices each corresponding to a word in the document (input)
- `W`: a list of weight matrices connecting each part of the network, e.g. for a network with a hidden and an output layer: W[0] is the weight matrix that connects the input to the first hidden layer, W[1] is the weight matrix that connects the hidden layer to the output layer.
- `dropout_rate`: the dropout rate that is used to generate a random dropout mask vector applied after each hidden layer for regularisation.

and returns:

- `out_vals`: a dictionary of output values from each layer: h (the vector before the activation function), a (the resulting vector after passing h from the activation function), its dropout mask vector; and the prediction vector (probability for each class) from the output layer.

In [110]:
def forward_pass(x, W, dropout_rate=0.2):
    out_vals = {}
    h_vecs = []
    a_vecs = []
    out_vals['a'] = np.array([x])
    dropout_vecs = []
    x_vec = np.zeros((len(x), len(W[0])))  # create a matrix to hold the word embeddings
    for i, w_idx in enumerate(x):
        if w_idx in vocab:
            x_vec[i, vocab[w_idx]] = 1
    # one-hot encode each word in the document
    h = x_vec.dot(W[0])  # compute the hidden layer before activation
    h_vecs.append(h)
    a = relu(h)  # apply the activation function
    dropout_vec = dropout_mask((1, a.shape[1]), dropout_rate)  # compute the dropout mask vector
    a = a * dropout_vec  # apply dropout to the output of the activation function
    a_vecs.append(a)
    dropout_vecs.append(dropout_vec)
    for i in range(1, len(W)):  # process any subsequent hidden layers
        h = a.dot(W[i])
        h_vecs.append(h)
        a = relu(h)
        dropout_vec = dropout_mask((1, a.shape[1]), dropout_rate)
        a = a * dropout_vec
        a_vecs.append(a)
        dropout_vecs.append(dropout_vec)
    out_vals['h_vecs'] = h_vecs
    out_vals['a_vecs'] = a_vecs
    out_vals['dropout_vecs'] = dropout_vecs
    if len(W) < 1:
        raise ValueError("W list should contain at least one weight matrix")
    elif len(W) == 1:
        y_pred = a.dot(W[0])
    else:
        y_pred = a.dot(W[1].T)  # compute the predicted class probabilities
    out_vals['y_pred'] = y_pred
    return out_vals


The `backward_pass` function computes the gradients and updates the weights for each matrix in the network from the output to the input. It takes as input 

- `x`: a list of vocabulary indices each corresponding to a word in the document (input)
- `y`: the true label
- `W`: a list of weight matrices connecting each part of the network, e.g. for a network with a hidden and an output layer: W[0] is the weight matrix that connects the input to the first hidden layer, W[1] is the weight matrix that connects the hidden layer to the output layer.
- `out_vals`: a dictionary of output values from a forward pass.
- `learning_rate`: the learning rate for updating the weights.
- `freeze_emb`: boolean value indicating whether the embedding weights will be updated.

and returns:

- `W`: the updated weights of the network.

Hint: the gradients on the output layer are similar to the multiclass logistic regression.

In [129]:
def backward_pass(x, y, W, out_vals, learning_rate, freeze_emb=False):
    # Compute the gradients on the output layer
    delta = out_vals['y_pred'] - y
    grad_out = np.dot(out_vals['a_vecs'][-1].T, delta)
    # Backpropagate the error and compute the gradients on the hidden layers
    deltas = [delta]
    grads = [grad_out]
    for i in range(len(W) - 2, -1, -1):
        delta = np.dot(deltas[-1], W[i+1]) * relu_derivative(out_vals['h_vecs'][i])
        deltas.append(delta)
        grad = np.dot(out_vals['a_vecs'][i].T, delta)
        grads.append(grad)

    # Reverse the gradients and deltas lists to match the order of the weight matrices
    grads.reverse()
    deltas.reverse()

    # Compute the gradients on the embedding layer if it's not frozen
    if not freeze_emb:
        delta_emb = np.zeros_like(W[0])
        for i, w_idx in enumerate(x):
            if w_idx in vocab:
                delta_emb[vocab[w_idx]] += np.dot(deltas[0], W[0][i].reshape(1,-1))
        grad_emb = delta_emb
        grads[0] = grad_emb

    # Update the weights of the network
    for i in range(len(W)):
        W[i] -= learning_rate * grads[i]

    return W


Finally you need to modify SGD to support back-propagation by using the `forward_pass` and `backward_pass` functions.

The `SGD` function takes as input:

- `X_tr`: array of training data (vectors)
- `Y_tr`: labels of `X_tr`
- `W`: the weights of the network (dictionary)
- `X_dev`: array of development (i.e. validation) data (vectors)
- `Y_dev`: labels of `X_dev`
- `lr`: learning rate
- `dropout`: regularisation strength
- `epochs`: number of full passes over the training data
- `tolerance`: stop training if the difference between the current and previous validation loss is smaller than a threshold
- `freeze_emb`: boolean value indicating whether the embedding weights will be updated (to be used by the backward pass function).
- `print_progress`: flag for printing the training progress (train/validation loss)


and returns:

- `weights`: the weights learned
- `training_loss_history`: an array with the average losses of the whole training set after each epoch
- `validation_loss_history`: an array with the average losses of the whole development set after each epoch

In [127]:
def SGD(X_tr, Y_tr, W, X_dev=[], Y_dev=[], lr=0.001, dropout=0.2, epochs=5, 
        tolerance=0.001, freeze_emb=False, print_progress=True):

    # Initialize loss history
    training_loss_history = []
    validation_loss_history = []

    # Make a copy of the training data
    X_tr_copy = np.copy(X_tr)

    # Iterate over epochs
    for epoch in range(epochs):

        # Shuffle the training data for each epoch
        idxs = np.random.permutation(len(X_tr_copy))
        X_tr_copy = X_tr_copy[idxs]
        Y_tr = Y_tr[idxs]

        # Initialize epoch losses
        epoch_training_loss = 0
        epoch_validation_loss = 0

        # Iterate over training data
        for i, (x, y) in enumerate(zip(X_tr_copy, Y_tr)):

            # Perform forward pass
            out_vals = forward_pass(x, W, dropout)

            # Compute loss and gradients with respect to output
            scores = out_vals['a'][-1]
            exp_scores = np.exp(scores)
            probs = exp_scores / np.sum(exp_scores, axis=0, keepdims=True)
            correct_logprobs = -np.log(probs[y])

            # Compute gradients through the network with backward pass
            dW = backward_pass(x, y, W, out_vals, lr, freeze_emb)

            # Update weights
            for j in range(len(W)):
                W[j] -= lr * dW[j]

            # Update epoch loss
            epoch_training_loss += correct_logprobs

        # Compute average epoch losses
        epoch_training_loss /= len(X_tr_copy)
        training_loss_history.append(epoch_training_loss)

        # Compute validation loss
        if len(X_dev) > 0:
            for x, y in zip(X_dev, Y_dev):
                out_vals = forward_pass(x, W, dropout)
                scores = out_vals['a'][-1]
                exp_scores = np.exp(scores)
                probs = exp_scores / np.sum(exp_scores, axis=0, keepdims=True)
                epoch_validation_loss += -np.log(probs[y])
            epoch_validation_loss /= len(X_dev)
            validation_loss_history.append(epoch_validation_loss)

        # Print progress
        if print_progress:
            print("Epoch %d: training loss=%.4f, validation loss=%.4f" % (epoch+1, epoch_training_loss, epoch_validation_loss))

        # Check for early stopping
        if len(validation_loss_history) >= 2 and abs(validation_loss_history[-1] - validation_loss_history[-2]) < tolerance:
            break

    return W, training_loss_history, validation_loss_history


Now you are ready to train and evaluate your neural net. First, you need to define your network using the `network_weights` function followed by SGD with backprop:

In [130]:
W = network_weights(vocab_size=len(vocab),embedding_dim=300,
                    hidden_dim=[], num_classes=3)

for i in range(len(W)):
    print('Shape W'+str(i), W[i].shape)
X_tr = train_indices
Y_tr = np.array(train_labels)
W, loss_tr, dev_loss = SGD(X_tr, Y_tr,
                            W,
                            X_dev=dev_indices, 
                            Y_dev=np.array(dev_labels),
                            lr=0.001, 
                            dropout=0.2,
                            freeze_emb=False,
                            tolerance=0.01,
                            epochs=100)


Shape W0 (10000, 300)
Shape W1 (300, 3)


C:\Users\86158\AppData\Local\Temp\ipykernel_17852\562675631.py:31: RuntimeWarning: overflow encountered in exp
  exp_scores = np.exp(scores)
C:\Users\86158\AppData\Local\Temp\ipykernel_17852\562675631.py:32: RuntimeWarning: invalid value encountered in true_divide
  probs = exp_scores / np.sum(exp_scores, axis=0, keepdims=True)


ValueError: operands could not be broadcast together with shapes (11,3) (11,300) 

Plot the learning process:

Compute accuracy, precision, recall and F1-Score:

In [10]:
preds_te = [np.argmax(forward_pass(x, W, dropout_rate=0.0)['y']) 
            for x,y in zip(X_te,Y_te)]

print('Accuracy:', accuracy_score(Y_te,preds_te))
print('Precision:', precision_score(Y_te,preds_te,average='macro'))
print('Recall:', recall_score(Y_te,preds_te,average='macro'))
print('F1-Score:', f1_score(Y_te,preds_te,average='macro'))

### Discuss how did you choose model hyperparameters ? 

# Use Pre-trained Embeddings

Now re-train the network using GloVe pre-trained embeddings. You need to modify the `backward_pass` function above to stop computing gradients and updating weights of the embedding matrix.

Use the function below to obtain the embedding martix for your vocabulary. Generally, that should work without any problem. If you get errors, you can modify it.

In [32]:
def get_glove_embeddings(f_zip, f_txt, word2id, emb_size=300):
    
    w_emb = np.zeros((len(word2id), emb_size))
    
    with zipfile.ZipFile(f_zip) as z:
        with z.open(f_txt) as f:
            for line in f:
                line = line.decode('utf-8')
                word = line.split()[0]
                     
                if word in vocab:
                    emb = np.array(line.strip('\n').split()[1:]).astype(np.float32)
                    w_emb[word2id[word]] +=emb
    return w_emb

In [33]:
w_glove = get_glove_embeddings("glove.840B.300d.zip","glove.840B.300d.txt",word2id)

First, initialise the weights of your network using the `network_weights` function. Second, replace the weigths of the embedding matrix with `w_glove`. Finally, train the network by freezing the embedding weights: 

In [14]:
preds_te = [np.argmax(forward_pass(x, W, dropout_rate=0.0)['y']) 
            for x,y in zip(X_te,Y_te)]

print('Accuracy:', accuracy_score(Y_te,preds_te))
print('Precision:', precision_score(Y_te,preds_te,average='macro'))
print('Recall:', recall_score(Y_te,preds_te,average='macro'))
print('F1-Score:', f1_score(Y_te,preds_te,average='macro'))

### Discuss how did you choose model hyperparameters ? 

# Extend to support deeper architectures 

Extend the network to support back-propagation for more hidden layers. You need to modify the `backward_pass` function above to compute gradients and update the weights between intermediate hidden layers. Finally, train and evaluate a network with a deeper architecture. Do deeper architectures increase performance?

In [13]:
preds_te = [np.argmax(forward_pass(x, W, dropout_rate=0.0)['y']) 
            for x,y in zip(X_te,Y_te)]

print('Accuracy:', accuracy_score(Y_te,preds_te))
print('Precision:', precision_score(Y_te,preds_te,average='macro'))
print('Recall:', recall_score(Y_te,preds_te,average='macro'))
print('F1-Score:', f1_score(Y_te,preds_te,average='macro'))

### Discuss how did you choose model hyperparameters ? 

## Full Results

Add your final results here:

| Model | Precision  | Recall  | F1-Score  | Accuracy
|:-:|:-:|:-:|:-:|:-:|
| Average Embedding  |   |   |   |   |
| Average Embedding (Pre-trained)  |   |   |   |   |
| Average Embedding (Pre-trained) + X hidden layers    |   |   |   |   |


Please discuss why your best performing model is better than the rest and provide a bried error analaysis.